## Data processing and Evaluation
This notebook is for the purpose of preparing the task 2 input data from output of task 1. 

#### Step 1: Load the data for task 1

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd '/content/drive/MyDrive/TU_Delft_DST/crowd_computing/crowd-sorcerers'

/content/drive/MyDrive/TU_Delft_DST/crowd_computing/crowd-sorcerers


In [51]:
#Uncomment the following lines if running this notebook for the first time


# !git clone https://github.com/nstruharova/crowd-sorcerers.git
# %cd crowd-sorcerers/
#!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 5 (delta 1), reused 5 (delta 1), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/nstruharova/crowd-sorcerers
   6c4927c..f998eba  main       -> origin/main
Updating 6c4927c..f998eba
Fast-forward
 output_task2/ChatMessages-2022-06-13.csv |  83 +++
 output_task2/wsd_app_2022-06-13.csv      | 841 +++++++++++++++++++++++++++++++
 2 files changed, 924 insertions(+)
 create mode 100644 output_task2/ChatMessages-2022-06-13.csv
 create mode 100644 output_task2/wsd_app_2022-06-13.csv


In [3]:
#The output of this folder should be './crowd-sorcerers'
%pwd

'/content/drive/MyDrive/TU_Delft_DST/crowd_computing/crowd-sorcerers'

In [4]:
#Creating the json file from the tsv file generated from the output of task 1 on toloka
import json
def tsv2json(input_file,output_file):
    arr = []
    file = open(input_file, 'r')
    a = file.readline()
    titles = [t.strip() for t in a.split('\t')]
    for line in file:
        d = {}
        for t, f in zip(titles, line.split('\t')):
            
            d[t] = f.strip()
              
        arr.append(d)
          
    with open(output_file, 'w', encoding='utf-8') as output_file:
        output_file.write(json.dumps(arr, indent=4))


In [5]:
tsv2json('task 1/results/res_10_percent.tsv', 'task 1/results/res_10_percent.json')

In [6]:
f = open('task 1/results/res_10_percent.json')
data = json.load(f)

In [8]:
import pandas as pd
df_task1 = pd.DataFrame(data)
df_task1.head(2)

,INPUT:id,INPUT:sense,INPUT:phrase,INPUT:sentence,OUTPUT:answer,GOLDEN:answer,HINT:text,HINT:default_language,ASSIGNMENT:link,ASSIGNMENT:task_id,ASSIGNMENT:assignment_id,ASSIGNMENT:worker_id,ASSIGNMENT:status,ASSIGNMENT:started
0,8,"""""""the products of human creativity; works of ...",art,It is easy to see why the ancient art is on th...,"""""""the creation of beautiful or significant th...",,,,https://toloka.yandex.ru/task/33830683/0002043...,000204371b--62a099c03e94fd20f6228d76,000204371b--62a09e4c996da35cd762332f,a5142f3e65e89f0d294e14c5bd09ac2f,APPROVED,2022-06-08T13:04:12.120
1,282,"""""""suitable to or characteristic of drama"""",""""...",dramatic,"It flopped all around, real dramatic!","""""""pertaining to or characteristic of drama""""""",,,,https://toloka.yandex.ru/task/33830683/0002043...,000204371b--62a099c03e94fd20f6228e88,000204371b--62a09e4c996da35cd762332f,a5142f3e65e89f0d294e14c5bd09ac2f,APPROVED,2022-06-08T13:04:12.120


#### Step 2: Extract and format the ground truth data 

In [9]:
# The current code is just a proof of concept, so the ground truth contains only 10 most confusing phrases. 
# the format of the gold_data here is: for one ambiguous phrase we have ['the_ambiguous phrase', 'correct_sense', 'input_sentence']
gold_data = [['challenges','questioning a statement and demanding an explanation', 'Kagan served as solicitor general in the Obama administration when the first legal challenges to the law were brought at the trial court level.'],
             ['nodded', 'lower and raise the head, as to indicate assent or agreement or confirmation', 'The man closed his eyes and nodded.'],
             ['nodded', 'express or signify by nodding', 'Five minutes later, before Haney could make his break, the stranger stood up and nodded farewell.'],
             ['nodded', 'express or signify by nodding', 'Pete nodded understandingly.'],
             ['equity', 'conformity with rules or standards', 'Whether they are or are not underpaid is a problem of equity; it is not an educational problem.'],
             ['equity', 'the ownership interest of shareholders in a corporation', ' If you were especially helpful in a corrupt scheme you received not just cash in a bag, but equity.'],
             ['workers', 'a person who works at a specific occupation', 'Fears about the impact of immigrants are based on the notion that they are liable to replace native workers, in particular unskilled ones, exerting downward pressure on wages.'],
             ['workers', 'a person who works at a specific occupation', 'The gender and ethnic backgrounds of workers and how these relate to age discrimination are examined.The gender and ethnic backgrounds of workers and how these relate to age discrimination are examined.'],
             ['individual', 'being or characteristic of a single thing or person','The individual voter \'s standing consequently will be enhanced through lower taxes.'],
             ['descending', 'move downward and lower, but not necessarily all the way', 'Only when one is ascending-- or in our case descending a tad trop rapidement-- does one feel, well, airborne in a picnic basket.']]


#### Step 3: Prepare the list of task 1 results relevant for evaluation

In [10]:
# Checks if an input phrase has gold data available. Only those output phrases will be filtered for which we have the the ground truth values 
def inGold(task2data, golddata):
  for gold in golddata:
    if (task2data[0] == gold[0]) and (task2data[2] == gold[2]):
      return True
  return False

In [18]:
#Compares task data output to gold data
def compareData(taskdata, golddata):
  for gold in golddata:
    if (taskdata[0] == gold[0]) and (taskdata[1] == gold[1]) and (taskdata[2] == gold[2]):
      return True
  return False

In [19]:
# Filtering the list of phrases for which we have the ground values
list_of_pred_task1 = []
for idx, row in df_task1.iterrows():
  ans_str = row['OUTPUT:answer'].strip('"')
  list_of_pred_task1.append([row['INPUT:phrase'],ans_str,row['INPUT:sentence']])
filtered_list_pred_task1 = []
for ans in list_of_pred_task1:
  if inGold(ans, gold_data):
    filtered_list_pred_task1.append(ans)

#### Step 4: Evaluate **task 1** for confusing phrases 

In [20]:
number_of_correct_responses_task1 = 0
for pred in filtered_list_pred_task1:  
  if compareData(pred, gold_data):
    number_of_correct_responses_task1 += 1
print(number_of_correct_responses_task1/len(filtered_list_pred_task1))

0.3333333333333333


#### Step 5: Load data for **task 2**

In [21]:
df_wsd_game2 = pd.read_csv("output_task2/wsd_app_2022-06-13.csv")
df_wsd_game2.head(5)
df_relevant_answers = df_wsd_game2[df_wsd_game2['player.decision'].notna()]

In [22]:
# df_relevant_answers = df_wsd_game.loc[df_wsd_game['participant._current_app_name'] == 'last']
new_df_data = []
for senses_concat in df_relevant_answers['player.senses']:
  new_df_list = senses_concat.split(sep=";;")
  new_df_data.append(new_df_list)
df_relevant_answers['player.sensesList'] = new_df_data
df_relevant_answers.head(5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,participant.id_in_session,participant.code,participant.label,participant._is_bot,participant._index_in_pages,participant._max_page_index,participant._current_app_name,participant._current_page_name,participant.time_started,participant.visited,...,group.is_eventual_agreement,group.is_golden_label,subsession.round_number,session.code,session.label,session.mturk_HITId,session.mturk_HITGroupId,session.comment,session.is_demo,player.sensesList
0,1,l81l91fj,00020551b2--62a70d75111d247825ed60ce,0,92,95,wsd_app,FinalResults,2022-06-13 10:12:09.495602+00:00,1,...,0.0,NaN,1,v8jw39uu,NaN,NaN,NaN,NaN,0,"[a demanding or stimulating situation, a call ..."
1,2,duey5jmx,00020551b2--62a70d7993820275f9e6511f,0,95,95,last,Last,2022-06-13 10:12:18.424442+00:00,1,...,0.0,NaN,1,v8jw39uu,NaN,NaN,NaN,NaN,0,"[a demanding or stimulating situation, a call ..."
2,3,kb4xh90w,00020551b2--62a70d853e4bbc46d7b63ed5,0,92,95,wsd_app,FinalResults,2022-06-13 10:12:28.742035+00:00,1,...,0.0,NaN,1,v8jw39uu,NaN,NaN,NaN,NaN,0,"[a demanding or stimulating situation, a call ..."
3,4,dh34bcev,00020551b2--62a70d79037a78554529687d,0,95,95,last,Last,2022-06-13 10:12:30.388528+00:00,1,...,0.0,NaN,1,v8jw39uu,NaN,NaN,NaN,NaN,0,"[a demanding or stimulating situation, a call ..."
5,6,wusx1aqt,00020551b2--62a70d81cd14e60843e78de3,0,92,95,wsd_app,FinalResults,2022-06-13 10:12:33.761728+00:00,1,...,0.0,NaN,1,v8jw39uu,NaN,NaN,NaN,NaN,0,"[a demanding or stimulating situation, a call ..."


#### Step 6: Preparing the task 2 data for evaluation

In [25]:
list_of_pred_task2 = []
df_relevant_answers['player.decision'] = df_relevant_answers['player.decision'].astype('int')
for idx, row in df_relevant_answers.iterrows():
  pred_sense_idx = row['player.decision']
  list_of_pred_task2.append([row['player.phrase'],row['player.sensesList'][pred_sense_idx], row['player.sentence']])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Step 7: Evaluate **task 2** for confusing phrases 

In [24]:
number_of_correct_responses = 0
for pred in list_of_pred_task2:  
  if compareData(pred, gold_data):
    number_of_correct_responses += 1
print(number_of_correct_responses/len(list_of_pred_task2))

0.235


## Pre-processing data for Task 2

In [26]:
#Get a list of unique phrases from the df
list_of_unique_phrases = df_task1['INPUT:phrase'].unique()


#Get probabilities for answers for each unique sense
list_dict_diagreed_phrases = []
for phrase in list_of_unique_phrases:
  df_phrase = df_task1.loc[df_task1['INPUT:phrase'] == phrase]
  list_of_possible_ans = df_phrase['OUTPUT:answer'].unique()
  length = len(df_phrase)
  list_of_probability_possible_ans = []
  for ans in list_of_possible_ans:
    df_ans = df_phrase.loc[df_phrase['OUTPUT:answer'] == ans]
    round((len(df_ans)/length),2)
    list_of_probability_possible_ans.append(round((len(df_ans)/length),2))
  list_dict_diagreed_phrases.append({phrase:sorted(list_of_probability_possible_ans, reverse=True)})
list_dict_diagreed_phrases[0]

{'art': [0.78, 0.11, 0.11]}

In [27]:
#Isolate a list of most confusing phrases on the basis of the margin error 
dict_margin_error = {}
for phrase_dict in list_dict_diagreed_phrases:
  for k in phrase_dict:
    if len(phrase_dict[k]) > 1:
      highest_prob_diff =  round((phrase_dict[k][0] -  phrase_dict[k][1]),2)
      dict_margin_error[k]=highest_prob_diff

sorted_dict_margin_error = dict(sorted(dict_margin_error.items(), key=lambda item: item[1]))
list_confusing_phrases = []
for key in sorted_dict_margin_error:
  if sorted_dict_margin_error[key] == 0.0:
    list_confusing_phrases.append(key)

list_confusing_phrases

['challenges',
 'nodded',
 'equity',
 'workers',
 'individual',
 'descending',
 'European',
 'generalizations',
 'appeals',
 'matters',
 'securities',
 'theory',
 'aware',
 'leaned',
 'At the same time',
 'power',
 'bothered',
 'saluting']

In [28]:
#CHECK PROBABILITY SORTING
df_damage = df_task1.loc[df_task1['INPUT:phrase'] == 'descending']
list_of_possible_ans = df_damage['OUTPUT:answer'].unique()
length = len(df_damage)
list_of_probability_possible_ans = []
for ans in list_of_possible_ans:
  df_ans = df_damage.loc[df_damage['OUTPUT:answer'] == ans]
  print(round((len(df_ans)/length),2))

0.38
0.38
0.25


In [30]:
# #Get df with only confusing 
f2 = open('task 1/toloka_data.json')
phrase_data = json.load(f2)
# phrase_data["data"][0]["phrase"]
confusing_data = []
for confusing_phrase in list_confusing_phrases:
  for idx, phrase_dict in enumerate(phrase_data["data"]):
    if phrase_data["data"][idx]["phrase"] == confusing_phrase:
      confusing_data.append(phrase_data["data"][idx])

In [ ]:
with open('confusing_phrases.json', 'w') as fout:
    json.dump(confusing_data, fout, indent=4, sort_keys=True)